Auto-complete or Text filler is one of the outstanding features in search bars available on various search engines and websites. It's a must to have feature on such sites because when users starts entering their searc keywords, they exactly don't know the best best word or combination of words required to produce the best search result.

Here, we are going to look into the most easiest method for auto complete. Previously, we've looked into BOW and TF-IDF techniques for feature extraction from text based data, now in this kernel you'll see a practicle usecase of chars and words based N-Grams feature creation. Even though we won't be training any ML or RNN model to solve our problem, we will be producing a decent result using a simple python and text processing techniques.

Let's get started

In [1]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

Mainly N-grams models can be implemented in two ways:
    - Character based N-grams
    - Word based N-grams

Let's look at them one by one

## Character based N-grams

In [2]:
# Scrap sentences from wikipedia on desired topic
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Deep_learning')
raw_html = raw_html.read()

# Read complete page paragraphs
article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')
article_text = ''

# concat all paragraphs
for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()
print(article_text[:1000])

deep learning  (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. learning can be supervised, semi-supervised or unsupervised.[1][2][3]
deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, machine vision, speech recognition, natural language processing, audio recognition, social network filtering, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[4][5][6]
artificial neural networks (anns) were inspired by information processing and distributed communication nodes in biological systems. anns have various differences from biological brains.  specifically, neural networks

In [3]:
# Cleaning sentences - remove everything other than characters, full-stop and a space.
article_text = re.sub(r'[^A-Za-z. ]', '', article_text)

In [4]:
len(article_text)

42971

Here idea is to find all n-gram continious char combinations and have their next occuring words (for best possible suggestion) ready in our ngram dictionary

In [5]:
# ngram length
n = 5

# dictionary, which will contain n-grams combinations as keys and next character, after each occurance, as an item in list 
ngrams = {}


# iterate over (length of chars-n)
for i in range(len(article_text)-n):
    # incrementaly find ngram sequences     
    seq = article_text[i:i+n]
    
    # insert in dictionary as key
    if seq not in ngrams.keys():
        # prepare empty list to insert next possible chars
        ngrams[seq] = []

    # push a next character
    ngrams[seq].append(article_text[i+n])

In [6]:
list(ngrams.items())[:2]

[('deep ',
  ['l',
   's',
   'n',
   'b',
   'i',
   'l',
   'l',
   'l',
   'l',
   'l',
   'g',
   'b',
   'b',
   'l',
   'l',
   'i',
   'l',
   'l',
   'l',
   'l',
   'm',
   'l',
   'l',
   'l',
   'l',
   's',
   'b',
   'n',
   'n',
   'n',
   'n',
   'f',
   'n',
   'l',
   'l',
   'n',
   'n',
   'l',
   'n',
   'n',
   'n',
   'l',
   'a',
   'l',
   'l',
   'b',
   'l',
   'l',
   'l',
   'l',
   'g',
   'n',
   'b',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l',
   'n',
   'l',
   'l',
   'l',
   'n',
   'n',
   'n',
   'a',
   'n',
   'n',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l',
   'n',
   'l',
   'l',
   'r',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l',
   'i',
   'l',
   'a',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l',
   'b',
   'n',
   'l',
   'l',
   'l',
   'l',
   't',
   't',
   'l',
   't',
   'l',
   'a',
   'l',
   'l',
   'l',
   'l',
   'l',
   ' ',
   'l',
   'l',
   'l',
   'l',
   'l',
   'l']),
 ('eep l',
  ['e',
   'e',


For a desired sequence or search *search_sequence*, find a complete suggestion of specified length suggestion_len.
 
To do that, we'll have to keep appending characters based on ngrams keys and item list. From item list, we'll be performing selection based on probality, i.e. mostly occured next character will have a better chance of getting picked.

In [7]:
# assuming very first ngram as a search sequence
search_sequence = article_text[0:n]

# init the suggestion output
output = search_sequence

# lenth of max chars in obtained suggestion
suggestion_len = 100

for i in range(suggestion_len):
    # break, if search sequence is not present in prepared ngram dictionary
    if search_sequence not in ngrams.keys():
        break
        
    # if ngram key is available, then find the list of next possible characters
    possible_chars = ngrams[search_sequence]
    print(f'possible_chars:{possible_chars}')
    
    # Randomly select the next possbile character, most common will have more chances
    next_char = possible_chars[random.randrange(len(possible_chars))]
    print(f'next_char:{next_char}')
    
    # Update the suggestion output
    output += next_char
    print(f'updated complete suggestion: {output}')
    
    # update the search sequence now, excluding first char as we move forward
    search_sequence = output[len(output)-n:len(output)]

possible_chars:['l', 's', 'n', 'b', 'i', 'l', 'l', 'l', 'l', 'l', 'g', 'b', 'b', 'l', 'l', 'i', 'l', 'l', 'l', 'l', 'm', 'l', 'l', 'l', 'l', 's', 'b', 'n', 'n', 'n', 'n', 'f', 'n', 'l', 'l', 'n', 'n', 'l', 'n', 'n', 'n', 'l', 'a', 'l', 'l', 'b', 'l', 'l', 'l', 'l', 'g', 'n', 'b', 'l', 'l', 'l', 'l', 'l', 'l', 'n', 'l', 'l', 'l', 'n', 'n', 'n', 'a', 'n', 'n', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'n', 'l', 'l', 'r', 'l', 'l', 'l', 'l', 'l', 'l', 'i', 'l', 'a', 'l', 'l', 'l', 'l', 'l', 'l', 'b', 'n', 'l', 'l', 'l', 'l', 't', 't', 'l', 't', 'l', 'a', 'l', 'l', 'l', 'l', 'l', ' ', 'l', 'l', 'l', 'l', 'l', 'l']
next_char:l
updated complete suggestion: deep l
possible_chars:['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 

In [8]:
print(f'Search sequence: ',article_text[0:n])
print(f'\nSuggestion: {output}')

Search sequence:  deep 

Suggestion: deep learning to imagenet victory computation when used to defenderstandards realizing d object of one ti


First few words in this sentence will make a sense, but considering it as a whole won't make much sense. Considering, we haven't used any typical RNN sentence generator, this is a decent output.

## Words based N-grams

In Words N-grams, we follow the same approach, only difference is, here we consider a word as a single entity.

In [9]:
# ngram dictionary to keep the possible ngrams as keys and next occuring words as list items
ngrams = {}

# words in a single ngram
words = 3

# Word Tokenization
words_tokens = nltk.word_tokenize(article_text)

# Iterate over words tokens
for i in range(len(words_tokens)-words):
    # incrementaly find ngram word sequences  
    seq = ' '.join(words_tokens[i:i+words])
    print(seq)

    # insert in dictionary as key
    if  seq not in ngrams.keys():
        # prepare empty list to insert next possible chars
        ngrams[seq] = []

    # push a next word
    ngrams[seq].append(words_tokens[i+words])

deep learning also
learning also known
also known as
known as deep
as deep structured
deep structured learning
structured learning is
learning is part
is part of
part of a
of a broader
a broader family
broader family of
family of machine
of machine learning
machine learning methods
learning methods based
methods based on
based on artificial
on artificial neural
artificial neural networks
neural networks with
networks with representation
with representation learning
representation learning .
learning . learning
. learning can
learning can be
can be supervised
be supervised semisupervised
supervised semisupervised or
semisupervised or unsupervised.deeplearning
or unsupervised.deeplearning architectures
unsupervised.deeplearning architectures such
architectures such as
such as deep
as deep neural
deep neural networks
neural networks deep
networks deep belief
deep belief networks
belief networks recurrent
networks recurrent neural
recurrent neural networks
neural networks and
networks and 

In [10]:
list(ngrams.items())[:20]

[('deep learning also', ['known']),
 ('learning also known', ['as']),
 ('also known as', ['deep', 'a']),
 ('known as deep', ['structured']),
 ('as deep structured', ['learning']),
 ('deep structured learning', ['is']),
 ('structured learning is', ['part']),
 ('learning is part', ['of', 'of']),
 ('is part of', ['a', 'stateoftheart']),
 ('part of a', ['broader']),
 ('of a broader', ['family']),
 ('a broader family', ['of']),
 ('broader family of', ['machine']),
 ('family of machine', ['learning']),
 ('of machine learning', ['methods', 'algorithms', '.', 'to']),
 ('machine learning methods', ['based', '.']),
 ('learning methods based', ['on']),
 ('methods based on', ['artificial']),
 ('based on artificial', ['neural', 'neural']),
 ('on artificial neural', ['networks', 'networks'])]

In [11]:
# assuming very first ngram as a search sequence
search_sequence = article_text[0:n]

# init the suggestion output
output = search_sequence

# lenth of max chars in obtained suggestion
suggestion_len = 100

for i in range(suggestion_len):
    # break, if search sequence is not present in prepared ngram dictionary
    if search_sequence not in ngrams.keys():
        break
        
    # if ngram key is available, then find the list of next possible characters
    possible_chars = ngrams[search_sequence]
    print(f'possible_chars:{possible_chars}')
    
    # Randomly select the next possbile character, most common will have more chances
    next_char = possible_chars[random.randrange(len(possible_chars))]
    print(f'next_char:{next_char}')
    
    # Update the suggestion output
    output += next_char
    print(f'updated complete suggestion: {output}')
    
    # update the search sequence now, excluding first char as we move forward
    search_sequence = output[len(output)-n:len(output)]

In [12]:
# assuming very first ngram as a search sequence
search_sequence = ' '.join(words_tokens[0:words])

# init the suggestion output
output = search_sequence

# lenth of max words in obtained suggestion
for i in range(50):
    # break, if search sequence is not present in prepared ngram dictionary
    if search_sequence not in ngrams.keys():
        break
    
    # if ngram key is available, then find the list of next possible characters
    possible_words = ngrams[search_sequence]
    
    # Randomly select the next possbile character, most common will have more chances
    next_word = possible_words[random.randrange(len(possible_words))]

    # Update the suggestion output
    output += ' ' + next_word
    seq_words = nltk.word_tokenize(output)

    # update the search sequence now, excluding first char as we move forward
    search_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])

In [13]:
print(f'Search sequence: ',' '.join(words_tokens[0:words]))
print(f'\nSuggestion: {output}')

Search sequence:  deep learning also

Suggestion: deep learning also known as a weightless neural network composed of a layers selforganising feature extraction neural network module soft followed by a multilayer classification neural network module gsn which were independently trained . each layer in turn as an unsupervised restricted boltzmann machine then finetuning it using supervised backpropagation . the papers


You see, it's a lot better than char based N-Grams. Both N-Grams have their significance in different kind of problems.

Please also take a look at this amazing article by Usman Malik:
    https://stackabuse.com/python-for-nlp-developing-an-automatic-text-filler-using-n-grams/

<br>
Later we'll solve this auto-complete problem, using a large amount of data, using GRU and LSTM Models.